
# prepare E3SM model data for archiving (CU peta library, access via globus)

# 1) synthetic float output
# 2) full Eulerian output


In [1]:

import sys
sys.path.append('/global/homes/c/cnissen/scripts/seawater-3.3.4/seawater/')
sys.path.append("/global/homes/c/cnissen/scripts/python_gsw_py3/")
import os
import numpy as np
import xarray as xr
from netCDF4 import Dataset
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from matplotlib import cm
import seawater
#from seawater import dist
import seawater as sw
import matplotlib.path as mpath
from cartopy.util import add_cyclic_point
import matplotlib.gridspec as gridspec
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import (LongitudeFormatter, LatitudeFormatter,
                                LatitudeLocator)
import random
from numba import njit
from math import sin, cos, sqrt, atan2, radians
from gsw import SA_from_SP
from tqdm import tqdm
from statsmodels.stats.weightstats import DescrStatsW
import math


In [2]:
#----
# goal: make data more easily usable -> reduce file size
#    group variables by physics/biogeochemistry/biology
#    all variables: type float (not double)
#    keep only variables of interest (in original files, there are a lot of "technical" variables)
#    keep only those floats that are seeded in the open ocean >2000m (8943 floats)
#----
#
# CHECK
#   make sure that missing value is correctly defined
#   keep all global attributes of original file
#

In [3]:
#-----
# saving data here
#-----
savepath     = '/global/cfs/cdirs/m4003/cnissen/6year_run/ARCHIVE/'
# check existence of paths
if not os.path.exists(savepath):
    print ('Created '+savepath)
    os.makedirs(savepath)
    

In [4]:
####
# MESH INFO
# specifics for trajectory output
####

rad_to_deg = 180.0/np.pi
latlim = -45.0

path_mesh = '/global/cfs/cdirs/m4003/maltrud/'
meshID = 'EC30to60E2r2'
meshfile = xr. open_dataset(path_mesh+'ocean.'+meshID+'.210210.nc')
#print(meshfile)

lon  = meshfile['lonCell'].values*rad_to_deg
lat  = meshfile['latCell'].values*rad_to_deg
topo = meshfile['bottomDepth'].values
area = meshfile['areaCell'].values
zlevs            = meshfile['refBottomDepth'].values
layerThickness   = meshfile['layerThickness'].values
restingThickness = meshfile['restingThickness'].values

print(len(lon),'nodes in mesh')
print(topo.shape)
print(area.shape)
print('Min/Max lon:',np.min(lon),np.max(lon))
print('Min/Max lat:',np.min(lat),np.max(lat))
print('layerThickness.shape:',layerThickness.shape)
print('restingThickness.shape:',restingThickness.shape)

meshfile.close()

dd1 = np.argmin(np.abs(zlevs-2000))
print('Average below depth level',dd1,zlevs[dd1])
dz = np.diff(np.hstack((0,zlevs)))[dd1:]


236853 nodes in mesh
(236853,)
(236853,)
Min/Max lon: 0.0007300572350528742 359.997672445938
Min/Max lat: -78.53259417674468 89.94461290099375
layerThickness.shape: (1, 236853, 60)
restingThickness.shape: (236853, 60)
Average below depth level 45 2074.8740234375005


In [5]:
#----
# determine indices of floats seeded "deep"
#----
# kick out floats in shallow regions (these are not advected I think)
ind = np.where(zlevs<=2000)[0] # 2000 or 2200

path = '/global/cfs/cdirs/m4003/maltrud/6year/floats/'
year_list = ['0055']
for yy in tqdm(range(0,1)):
    print('Load year '+year_list[yy])
    file1 = 'floats.year'+year_list[yy]+'.nc'   
    data = xr.open_dataset(path+file1)
    no3_1   = data['particleColumnNO3'] #.values 

    if yy==0:
        # those that are not advected have the same position for all time steps; filter by using the first time step
        #aux = np.sum(np.isnan(no3_1[0,ind,:]),axis=0) # check if any of the depth levels shallower than 1100m is NaN
        #aux = np.sum(1*(no3_1[0,ind,:].values==-1),axis=0)
        aux = np.sum(1*(no3_1[0,ind,:]==-1),axis=0)
        ind_deep = np.where(aux==0)[0] # if it is, aux is >0; only keep those that are 0
        print(ind_deep.shape)
        

100%|██████████| 1/1 [00:00<00:00,  5.32it/s]

Load year 0055
(8739,)


In [11]:
#--
# choose precision of output
#--

format_data1 = 'f8' # for all variables involved in computation of carbonate chemistry
format_data2 = 'f4' # for all others


In [17]:
#----
# load daily float output -> PHYSICS, BIOLOGY or BIOGEOCHEMISTRY
#----

process_floats = False

save_netcdf = False

vari_string = 'BIOLOGY' # PHYSICS, BIOGEOCHEMISTRY, BIOLOGY

vari_list_phys = ['particleColumnTemperature','particleColumnSalinity','particleColumnVelMerid','particleColumnVelZonal',\
            'particle_MLD','particle_ifrac','particle_atmPressure']

vari_list_bio = ['particleColumnTotalPhytoC','particleColumnTotalPhytoChl','particleColumnZooC']

vari_list_bgc = ['particleColumnDIC','particleColumnALK','particleColumnNO3','particleColumnPO4','particleColumnSiO3',\
                'particleColumnO2','particleColumnO2_Consumption','particleColumnO2_Production',\
                'particle_CO2_gas_flux','particle_O2_gas_flux','particle_pCO2surface','particle_atmosphericCO2',\
                'particle_dpCO2']

list_2D_var = ['particle_MLD','particle_ifrac','particle_atmPressure',\
              'particle_CO2_gas_flux','particle_O2_gas_flux','particle_pCO2surface','particle_atmosphericCO2',\
                'particle_dpCO2']

list_vari_carbonate = ['particleColumnTemperature','particleColumnSalinity','particle_atmPressure',\
                        'particleColumnDIC','particleColumnALK','particleColumnPO4','particleColumnSiO3']

if vari_string in ['PHYSICS']:
    vari_string2 = 'physical'
    var_list = np.copy(vari_list_phys)
elif vari_string in ['BIOGEOCHEMISTRY']:
    vari_string2 = 'biogeochemical'
    var_list = np.copy(vari_list_bgc)
elif vari_string in ['BIOLOGY']:
    vari_string2 = 'biological'
    var_list = np.copy(vari_list_bio)

# needed for lat/lon in model
rad_to_deg = 180.0/np.pi

path = '/global/cfs/cdirs/m4003/maltrud/6year/floats/'
year_list  = ['0055','0056','0057','0058','0059','0060']
year_list2 = ['2012','2013','2014','2015','2016','2017']

if process_floats:
    for yy in range(0,len(year_list)):
        print('Load year '+year_list[yy])

        file1 =  'floats.year'+year_list[yy]+'.nc'  
        ff = xr.open_dataset(path+file1)

        lon_1   = ff['particleColumnLon'] #.values*rad_to_deg 
        lat_1   = ff['particleColumnLat'] #.values*rad_to_deg 
        lat_all = lat_1[:,0,:].values*rad_to_deg 
        lon_all = lon_1[:,0,:].values*rad_to_deg 
        # the part below is just to make sure there is no random zeros in the fields...
        lat_all[lat_all==0]=np.nan
        lon_all[lon_all==0]=np.nan
        # reduce to floats seeded in "deep" waters
        lat_all  = lat_all[:,ind_deep]
        lon_all  = lon_all[:,ind_deep]
        del lon_1,lat_1

        #---
        # create netcdf
        # add dimensions
        #---
        if save_netcdf:
            netcdf_name       = 'E3SMv2_synthetic_floats_'+vari_string+'_year_'+str(year_list2[yy])+'.nc' 
            script = '/global/homes/c/cnissen/scripts/plot_floats_E3SM_ARCHIVE_GMD_paper.ipynb'
            if not os.path.exists(savepath+netcdf_name):
                print ('Create file '+savepath+netcdf_name)
                w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
                # global arguments
                w_nc_fid.info = 'File containing all '+vari_string2+\
                        ' variables from synthetic floats in E3SMv2 for the year '+str(year_list2[yy])
                w_nc_fid.contact = 'cara.nissen@colorado.edu or nicole.lovenduski@colorado.edu or maltrud@lanl.gov'
                w_nc_fid.original_file = path+file1
                w_nc_fid.script = script
                w_nc_fid.meshfile = 'Mesh file underlying the seeding: '+path_mesh+'ocean.'+meshID+'.210210.nc'
                # create dimensions
                w_nc_fid.createDimension('nParticles', lon_all.shape[1]) 
                w_nc_fid.createDimension('nVertLevels', len(zlevs)) 
                w_nc_fid.createDimension('Time', lon_all.shape[0]) 
                # create variables for dimensions
                w_nc_var1 = w_nc_fid.createVariable('nVertLevels', format_data2,('nVertLevels'))
                w_nc_var1.note = 'bottom of vertical levels in m sampled by each float'
                w_nc_var1 = w_nc_fid.createVariable('Time', format_data2,('Time'))
                w_nc_var1.note = 'Day of the year (sampling at midnight GMT on each day)'
                w_nc_var1 = w_nc_fid.createVariable('lat', format_data2,('nParticles','Time'))
                w_nc_var1.note = 'Latitude in deg N at sampling'
                w_nc_var1 = w_nc_fid.createVariable('lon', format_data,('nParticles','Time'))
                w_nc_var1.note = 'Longitude in deg E at sampling (0-360°)'
                # fill dimension variables
                w_nc_fid.variables['nVertLevels'][:] = zlevs
                w_nc_fid.variables['Time'][:] = np.arange(1,lon_all.shape[0]+1)
                w_nc_fid.variables['lat'][:,:] = lat_all.transpose()
                w_nc_fid.variables['lon'][:,:] = lon_all.transpose()
                w_nc_fid.close()
            else:
                print ('File '+savepath+netcdf_name+' exists already, overwrite')

        for vv in tqdm(range(0,len(var_list))):
            print('variable: '+var_list[vv])

            data1 = ff[var_list[vv]]
            var_units = data1.attrs['units']
            long_name = data1.attrs['long_name']
            data_all = data1.values
            del data1

            ## set missing values to NaN (deep ocean layers) 
            data_all[data_all==-1]=np.nan
            # the part below is just to make sure there is no random zeros in the fields...
            data_all[data_all==0]=np.nan
            # reduce to floats seeded in "deep" waters
            if var_list[vv] in list_2D_var:
                data_all = data_all[:,ind_deep]
            else:
                data_all = data_all[:,:,ind_deep]

            #---
            # add variable to netcdf
            #---
            fv = -999
            if save_netcdf:
                w_nc_fid  = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC')   
                if var_list[vv] in list_vari_carbonate:
                    format_data = format_data1 # double precision
                else:
                    format_data = format_data2 # single precision
                print('Precision in output:',format_data)
                if var_list[vv] in list_2D_var:
                    w_nc_var1 = w_nc_fid.createVariable(var_list[vv], format_data,('nParticles','Time'),fill_value=fv)
                else:
                    w_nc_var1 = w_nc_fid.createVariable(var_list[vv], format_data,('nParticles','nVertLevels','Time'),fill_value=fv)
                w_nc_var1.units     = var_units
                w_nc_var1.long_name = long_name
                data_all[np.isnan(data_all)] = fv
                # write field
                if var_list[vv] in list_2D_var:
                    w_nc_fid.variables[var_list[vv]][:,:] = np.transpose(data_all,[1,0])
                else:
                    w_nc_fid.variables[var_list[vv]][:,:,:] = np.transpose(data_all,[2,1,0])
                w_nc_fid.close()
                print('Saved variable',var_list[vv])

            del data_all
        ff.close()
        del lon_all,lat_all

    print ('done')


Load year 0055
Create file /global/cfs/cdirs/m4003/cnissen/6year_run/ARCHIVE/E3SMv2_synthetic_floats_BIOLOGY_year_2012.nc


  0%|          | 0/3 [00:00<?, ?it/s]

variable: particleColumnTotalPhytoC
Precision in output: f4


 33%|███▎      | 1/3 [00:14<00:28, 14.30s/it]

Saved variable particleColumnTotalPhytoC
variable: particleColumnTotalPhytoChl
Precision in output: f4


...

Saved variable particleColumnTotalPhytoC
variable: particleColumnTotalPhytoChl
Precision in output: f4


 67%|██████▋   | 2/3 [00:25<00:12, 12.52s/it]

Saved variable particleColumnTotalPhytoChl
variable: particleColumnZooC
Precision in output: f4


100%|██████████| 3/3 [00:35<00:00, 11.71s/it]

Saved variable particleColumnZooC
done


In [ ]:
print(lon.shape)

In [ ]:
#----
# EULERIAN OUTPUT, MONTHLY
#----

In [20]:
#----
# load monthly Eulerian output -> PHYSICS, BIOLOGY or BIOGEOCHEMISTRY
#----

process_eulerian_monthly = False

save_netcdf = True
vari_string = 'BIOGEOCHEMISTRY' # PHYSICS, BIOGEOCHEMISTRY, BIOLOGY

vari_list_phys = ['timeMonthly_avg_activeTracers_temperature','timeMonthly_avg_activeTracers_salinity',\
                  'timeMonthly_avg_velocityMeridional','timeMonthly_avg_velocityZonal',\
            'timeMonthly_avg_dThreshMLD','timeMonthly_avg_iceAreaCell','timeMonthly_avg_atmosphericPressure']

vari_list_bio = ['timeMonthly_avg_ecosysTracers_diatC','timeMonthly_avg_ecosysTracers_diazC',\
                 'timeMonthly_avg_ecosysTracers_spC',\
                 'timeMonthly_avg_ecosysTracers_diatChl','timeMonthly_avg_ecosysTracers_diazChl',\
                 'timeMonthly_avg_ecosysTracers_spChl','timeMonthly_avg_ecosysTracers_zooC']

vari_list_bgc = ['timeMonthly_avg_ecosysTracers_DIC','timeMonthly_avg_ecosysTracers_ALK',\
                 'timeMonthly_avg_ecosysTracers_NO3','timeMonthly_avg_ecosysTracers_PO4',\
                 'timeMonthly_avg_ecosysTracers_SiO3',\
                'timeMonthly_avg_ecosysTracers_O2',\
                 'timeMonthly_avg_ecosys_diag_O2_CONSUMPTION','timeMonthly_avg_ecosys_diag_O2_PRODUCTION',\
                'timeMonthly_avg_CO2_gas_flux','timeMonthly_avg_ecosysTracersSurfaceFlux_O2SurfaceFlux',\
                 'timeMonthly_avg_ecosys_diag_pCO2surface','timeMonthly_avg_atmosphericCO2',\
                'timeMonthly_avg_ecosys_diag_dpCO2']

list_2D_var = ['timeMonthly_avg_dThreshMLD','timeMonthly_avg_iceAreaCell','timeMonthly_avg_atmosphericPressure',\
              'timeMonthly_avg_CO2_gas_flux','timeMonthly_avg_ecosysTracersSurfaceFlux_O2SurfaceFlux',\
               'timeMonthly_avg_ecosys_diag_pCO2surface','timeMonthly_avg_atmosphericCO2',\
                'timeMonthly_avg_ecosys_diag_dpCO2']

list_vari_carbonate = ['timeMonthly_avg_activeTracers_temperature','timeMonthly_avg_activeTracers_salinity',\
                       'timeMonthly_avg_atmosphericPressure',\
                        'timeMonthly_avg_ecosysTracers_DIC','timeMonthly_avg_ecosysTracers_ALK',\
                       'timeMonthly_avg_ecosysTracers_PO4','timeMonthly_avg_ecosysTracers_SiO3']

if vari_string in ['PHYSICS']:
    vari_string2 = 'physical'
    var_list = np.copy(vari_list_phys)
elif vari_string in ['BIOGEOCHEMISTRY']:
    vari_string2 = 'biogeochemical'
    var_list = np.copy(vari_list_bgc)
elif vari_string in ['BIOLOGY']:
    vari_string2 = 'biological'
    var_list = np.copy(vari_list_bio)

# needed for lat/lon in model
rad_to_deg = 180.0/np.pi

path = '/global/cfs/cdirs/m4003/maltrud/6year/monthlyEulerianAverages/'
year_list  = ['0055','0056','0057','0058','0059','0060']
year_list2 = ['2012','2013','2014','2015','2016','2017']

if process_eulerian_monthly:
    for yy in range(0,len(year_list)):
        print('Load year '+year_list[yy])

        file1 =  'monthlyAverageEulerianFields.year'+year_list[yy]+'.nc'  
        ff = xr.open_dataset(path+file1)

        #---
        # create netcdf
        # add dimensions
        #---
        if save_netcdf:
            netcdf_name       = 'E3SMv2_Eulerian_output_monthly_'+vari_string+'_year_'+str(year_list2[yy])+'.nc' 
            script = '/global/homes/c/cnissen/scripts/plot_floats_E3SM_ARCHIVE_GMD_paper.ipynb'
            if not os.path.exists(savepath+netcdf_name):
                print ('Create file '+savepath+netcdf_name)
                w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
                # global arguments
                w_nc_fid.info = 'File containing monthly Eulerian model output for all '+vari_string2+\
                        ' variables in E3SMv2 for the year '+str(year_list2[yy])
                w_nc_fid.contact = 'cara.nissen@colorado.edu or nicole.lovenduski@colorado.edu or maltrud@lanl.gov'
                w_nc_fid.original_file = path+file1
                w_nc_fid.script = script
                w_nc_fid.meshfile = 'Mesh file: '+path_mesh+'ocean.'+meshID+'.210210.nc'
                # create dimensions
                w_nc_fid.createDimension('nCells', lon.shape[0]) 
                w_nc_fid.createDimension('nVertLevels', len(zlevs)) 
                w_nc_fid.createDimension('Time', 12) # MONTHLY
                # create variables for dimensions
                w_nc_var1 = w_nc_fid.createVariable('nVertLevels', format_data2,('nVertLevels'))
                w_nc_var1.note = 'bottom of vertical levels in m'
                w_nc_var1 = w_nc_fid.createVariable('lat', format_data2,('nCells'))
                w_nc_var1.note = 'Latitude in deg N'
                w_nc_var1 = w_nc_fid.createVariable('lon', format_data2,('nCells'))
                w_nc_var1.note = 'Longitude in deg E (0-360°)'
                # fill dimension variables
                w_nc_fid.variables['nVertLevels'][:] = zlevs
                w_nc_fid.variables['lat'][:] = lat
                w_nc_fid.variables['lon'][:] = lon
                w_nc_fid.close()
            else:
                print ('File '+savepath+netcdf_name+' exists already, overwrite')

        for vv in tqdm(range(0,len(var_list))):
            print('variable: '+var_list[vv])
            
            if var_list[vv] in ['timeMonthly_avg_iceAreaCell']:
                file2 =  'monthlyAverageIceFraction.year'+year_list[yy]+'.nc'  
                ff1 = xr.open_dataset(path+file2)
                data1 = ff1[var_list[vv]]
                ff1.close()
                del file2
            else:
                data1 = ff[var_list[vv]]
            var_units = data1.attrs['units']
            long_name = data1.attrs['long_name']
            data_all = np.squeeze(data1.values)
            del data1
            
            #print(data_all.shape)

           # ## set missing values to NaN (deep ocean layers) 
           # data_all[data_all==-1]=np.nan
           # # the part below is just to make sure there is no random zeros in the fields...
           # data_all[data_all==0]=np.nan

            #---
            # add variable to netcdf
            #---
            fv = -999
            if save_netcdf:
                w_nc_fid  = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC') 
                if var_list[vv] in list_vari_carbonate:
                    format_data = format_data1 # double precision
                else:
                    format_data = format_data2 # single precision
                print('Precision in output:',format_data)
                if var_list[vv] in list_2D_var:
                    w_nc_var1 = w_nc_fid.createVariable(var_list[vv], format_data,('nCells','Time'),fill_value=fv)
                else:
                    w_nc_var1 = w_nc_fid.createVariable(var_list[vv], format_data,('nCells','nVertLevels','Time'),fill_value=fv)
                w_nc_var1.units     = var_units
                w_nc_var1.long_name = long_name
                data_all[np.isnan(data_all)] = fv
                data_all[data_all==0] = fv
                data_all[data_all<-999] = fv
                # write field
                if var_list[vv] in list_2D_var:
                    w_nc_fid.variables[var_list[vv]][:,:] = np.transpose(data_all,[1,0])
                else:
                    w_nc_fid.variables[var_list[vv]][:,:,:] = np.transpose(data_all,[1,2,0])
                w_nc_fid.close()
                print('Saved variable',var_list[vv])

            del data_all
        ff.close()

    print ('done')

    

Load year 0055
Create file /global/cfs/cdirs/m4003/cnissen/6year_run/ARCHIVE/E3SMv2_Eulerian_output_monthly_BIOGEOCHEMISTRY_year_2012.nc


  0%|          | 0/13 [00:00<?, ?it/s]

variable: timeMonthly_avg_ecosysTracers_DIC
Precision in output: f8


  8%|▊         | 1/13 [00:04<00:55,  4.59s/it]

Saved variable timeMonthly_avg_ecosysTracers_DIC
variable: timeMonthly_avg_ecosysTracers_ALK
Precision in output: f8


...

Saved variable timeMonthly_avg_ecosys_diag_O2_PRODUCTION
variable: timeMonthly_avg_CO2_gas_flux
Precision in output: f4
Saved variable timeMonthly_avg_CO2_gas_flux
variable: timeMonthly_avg_ecosysTracersSurfaceFlux_O2SurfaceFlux
Precision in output: f4


 92%|█████████▏| 12/13 [00:24<00:00,  1.35it/s]

Saved variable timeMonthly_avg_ecosysTracersSurfaceFlux_O2SurfaceFlux
variable: timeMonthly_avg_ecosys_diag_pCO2surface
Precision in output: f4
Saved variable timeMonthly_avg_ecosys_diag_pCO2surface
variable: timeMonthly_avg_atmosphericCO2
Precision in output: f4
Saved variable timeMonthly_avg_atmosphericCO2
variable: timeMonthly_avg_ecosys_diag_dpCO2
Precision in output: f4


100%|██████████| 13/13 [00:24<00:00,  1.88s/it]

Saved variable timeMonthly_avg_ecosys_diag_dpCO2
done


In [ ]:
#----
# EULERIAN OUTPUT, DAILY
#----

In [ ]:
#----
# load daily Eulerian output -> PHYSICS, BIOLOGY or BIOGEOCHEMISTRY
#----

process_eulerian_daily = True
ind_dd = 24 # only store upper XX depth levels (biomass is zero below)

save_netcdf = True
vari_string = 'BIOLOGY' # BIOLOGY only!

vari_list_bio = ['timeDaily_avg_ecosysTracers_zooC','timeDaily_avg_ecosysTracers_TotalPhytoC',\
                 'timeDaily_avg_ecosysTracers_TotalPhytoChl']
list_2D_var = []

if vari_string in ['PHYSICS']:
    vari_string2 = 'physical'
    var_list = np.copy(vari_list_phys)
elif vari_string in ['BIOGEOCHEMISTRY']:
    vari_string2 = 'biogeochemical'
    var_list = np.copy(vari_list_bgc)
elif vari_string in ['BIOLOGY']:
    vari_string2 = 'biological'
    var_list = np.copy(vari_list_bio)

# needed for lat/lon in model
rad_to_deg = 180.0/np.pi

path = '/global/cfs/cdirs/m4003/maltrud/6year/dailyEulerianAverages/'
year_list  = ['0055','0056','0057','0058','0059','0060']
year_list2 = ['2012','2013','2014','2015','2016','2017']

months = ['01','02','03','04','05','06','07','08','09','10','11','12']

if process_eulerian_daily:
    for yy in range(0,len(year_list)):
        print('Load year '+year_list[yy])
            
        #---
        # create netcdf
        # add dimensions
        #---
        if save_netcdf:
            netcdf_name       = 'E3SMv2_Eulerian_output_daily_'+vari_string+'_year_'+str(year_list2[yy])+'.nc' 
            script = '/global/homes/c/cnissen/scripts/plot_floats_E3SM_ARCHIVE_GMD_paper.ipynb'
            if not os.path.exists(savepath+netcdf_name):
                print ('Create file '+savepath+netcdf_name)
                w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
                # global arguments
                w_nc_fid.info = 'File containing daily Eulerian model output for all '+vari_string2+\
                        ' variables in E3SMv2 for the year '+str(year_list2[yy])
                w_nc_fid.contact = 'cara.nissen@colorado.edu or nicole.lovenduski@colorado.edu or maltrud@lanl.gov'
                w_nc_fid.original_file = path+file1
                w_nc_fid.script = script
                w_nc_fid.meshfile = 'Mesh file: '+path_mesh+'ocean.'+meshID+'.210210.nc'
                # create dimensions
                w_nc_fid.createDimension('nCells', lon.shape[0]) 
                w_nc_fid.createDimension('nVertLevels', len(zlevs[0:ind_dd])) 
                w_nc_fid.createDimension('Time', 365) # DAILY
                # create variables for dimensions
                w_nc_var1 = w_nc_fid.createVariable('nVertLevels', format_data2,('nVertLevels'))
                w_nc_var1.note = 'bottom of vertical levels in m'
                w_nc_var1 = w_nc_fid.createVariable('lat', format_data2,('nCells'))
                w_nc_var1.note = 'Latitude in deg N'
                w_nc_var1 = w_nc_fid.createVariable('lon', format_data2,('nCells'))
                w_nc_var1.note = 'Longitude in deg E (0-360°)'
                # fill dimension variables
                w_nc_fid.variables['nVertLevels'][:] = zlevs[0:ind_dd]
                w_nc_fid.variables['lat'][:] = lat
                w_nc_fid.variables['lon'][:] = lon
                w_nc_fid.close()
            else:
                print ('File '+savepath+netcdf_name+' exists already, overwrite')

        for vv in tqdm(range(0,len(var_list))):
            print('variable: '+var_list[vv])
            
            for mm in range(0,len(months)):
                print('Load month',months[mm])

                file1 = 'GMPAS-OIECO-JRA_r05_EC30to60E2r2_floats_chrys01.mpaso.hist.am.timeSeriesStatsDaily.'+\
                    year_list[yy]+'-'+months[mm]+'-01.Biomass.nc'

                ff = xr. open_dataset(path+file1)
                if vari_list_bio[vv] in ['timeDaily_avg_ecosysTracers_TotalPhytoC']: 
                    data1=ff['timeDaily_avg_ecosysTracers_diatC'].values.squeeze()
                    data2=ff['timeDaily_avg_ecosysTracers_diazC'].values.squeeze()
                    data3=ff['timeDaily_avg_ecosysTracers_spC'].values.squeeze()
                elif vari_list_bio[vv] in ['timeDaily_avg_ecosysTracers_TotalPhytoChl']: 
                    data1=ff['timeDaily_avg_ecosysTracers_diatChl'].values.squeeze()
                    data2=ff['timeDaily_avg_ecosysTracers_diazChl'].values.squeeze()
                    data3=ff['timeDaily_avg_ecosysTracers_spChl'].values.squeeze()
                elif vari_list_bio[vv] in ['timeDaily_avg_ecosysTracers_zooC']: 
                    data1=ff['timeDaily_avg_ecosysTracers_zooC'].values.squeeze()
                    data2=0
                    data3=0
                ff.close()
                aux = data1 + data2 + data3
                aux = aux[:,:,0:ind_dd] # only store upper XX depth levels
                del data1,data2,data3
                aux = np.ma.masked_where(aux<0,aux)

                if mm==0:
                    data_all = aux
                else:
                    data_all  = np.concatenate((data_all,aux))

                print(data_all.shape)
            
            if vari_list_bio[vv] in ['timeDaily_avg_ecosysTracers_TotalPhytoC']: 
                data1=ff['timeDaily_avg_ecosysTracers_diatC']
                long_name = 'total phytoplankton carbon'
            elif vari_list_bio[vv] in ['timeDaily_avg_ecosysTracers_TotalPhytoChl']: 
                data1=ff['timeDaily_avg_ecosysTracers_diatChl']
                long_name = 'total phytoplankton chlorophyll'
            elif vari_list_bio[vv] in ['timeDaily_avg_ecosysTracers_zooC']: 
                data1=ff['timeDaily_avg_ecosysTracers_zooC']
                long_name = 'zooplankton carbon'
            var_units = data1.attrs['units']
            #long_name = data1.attrs['long_name']
            print(var_units,long_name)
            del data1

            #---
            # add variable to netcdf
            #---
            fv = -999
            if save_netcdf:
                w_nc_fid  = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC')     
                if var_list[vv] in list_vari_carbonate:
                    format_data = format_data1 # double precision
                else:
                    format_data = format_data2 # single precision
                print('Precision in output:',format_data)
                if var_list[vv] in list_2D_var:
                    w_nc_var1 = w_nc_fid.createVariable(var_list[vv], format_data,('nCells','Time'),fill_value=fv)
                else:
                    w_nc_var1 = w_nc_fid.createVariable(var_list[vv], format_data,('nCells','nVertLevels','Time'),fill_value=fv)
                w_nc_var1.units     = var_units
                w_nc_var1.long_name = long_name
                data_all[np.isnan(data_all)] = fv
                data_all[data_all==0] = fv
                data_all[data_all<=-999] = fv
                # write field
                if var_list[vv] in list_2D_var:
                    w_nc_fid.variables[var_list[vv]][:,:] = np.transpose(data_all,[1,0])
                else:
                    w_nc_fid.variables[var_list[vv]][:,:,:] = np.transpose(data_all,[1,2,0])
                w_nc_fid.close()
                print('Saved variable',var_list[vv])

            del data_all
        ff.close()

    print ('done')

    
    

In [ ]:
print(np.transpose(data_all,[1,2,0]).shape)
print(data_all.shape)


In [ ]:
print(path+file1)